In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
import re
pd.options.display.max_columns = None
from fuzzywuzzy import process, fuzz
from collections import Counter
import calendar

import sys
sys.path.append("../")
import src.soporteAPIs as sa
sys.path.append("../")
import src.biblioteca as bb

# Importing pickle file

In [2]:
with open('../data/pickle/artist.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [3]:
tracks.head()

,url,artist_clean,track_clean
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [4]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.head()

,url,artist,track
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


I will split the artist column in order to split between main artist (`artist_0`) and possible featuring artist (`artist_1`, `artist_1`)

In [5]:
test = tracks['artist'].str.replace('tyler, the creator', 'tyler the creator').str.split(',', n=2, expand = True)
test

,0,1,2
0,migos,None,None
1,drake,None,None
2,the weeknd,daft punk,None
3,the chainsmokers,halsey,None
4,rae sremmurd,gucci mane,None
...,...,...,...
454434,fase yoda,None,None
454442,max,ali gatie,None
454443,cooper alan,rvshvd,None
454451,kali,None,None


In [6]:
tracks = pd.concat([tracks, test], axis = 1)
tracks.head()

,url,artist,track,0,1,2
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,None,None
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,the weeknd,daft punk,None
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,the chainsmokers,halsey,None
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,rae sremmurd,gucci mane,None


In [7]:
tracks.rename(columns={0:'artist_0', 1:'artist_1', 2:'artist_2'}, inplace=True)
tracks.head()

,url,artist,track,artist_0,artist_1,artist_2
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,None,None
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,the weeknd,daft punk,None
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,the chainsmokers,halsey,None
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,rae sremmurd,gucci mane,None


# lastfm
In this part of the process I will extract form the LastFM API all the data related to the tracks and artist genres.

First, I need to configure my credentials.

In [ ]:
load_dotenv()

In [ ]:
api = os.getenv("lastfm-id")
username = os.getenv("lastfm-user")

Testing to gather the data from the api with a function:

In [ ]:
res = sa.getLastFMData({
    'method': 'artist.getInfo',
    'artist':'Harry Styles',
})
res.json()['artist']['bio']

First I try on a slice of my dataframe

In [ ]:
tqdm.pandas()

test = tracks.sample(100)
test[['bio', 'artist_tag']] = test.progress_apply(lambda x: sa.getArtistInfo(x.artist_0), axis=1, result_type='expand')

In [ ]:
test.head()

This is for the whole dataframe:

In [ ]:
tqdm.pandas()

#test = tracks.sample(100)
tracks[['bio', 'artist_tag']] = tracks.progress_apply(lambda x: sa.getArtistInfo(x.artist_0), axis=1, result_type='expand')

In [ ]:
tracks.head()

Now I get the tags for each individual track:

In [ ]:
res = sa.getLastFMData({
    'method': 'track.getInfo',
    'artist':'Harry Styles',
    'track':'As It Was'
})
res.json()['track']['toptags']['tag']#['published']#[0]['name']

This is how I will get info for a slice of my dataframe:

In [ ]:
tqdm.pandas()

test = tracks.sample(100)
test[['track_tag', 'published_track']] = test.progress_apply(lambda x: sa.getTrackTags(x.artist_0, x.track), axis=1, result_type='expand')

In [ ]:
print(test['track_tag'].isnull().sum())
test.head()

In [ ]:
test.tail()

This is for the whole dataset:

In [ ]:
tqdm.pandas()

tracks[['track_tag', 'published_track']] = tracks.progress_apply(lambda x: sa.getTrackTags(x.artist_0, x.track), axis=1, result_type='expand')

In [ ]:
print(tracks['track_tag'].isnull().sum(), tracks['artist_tag'].isnull().sum())
tracks.head()

# Data Cleaning

In [80]:
with open('../data/pickle/lastfm_dump.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

Cleaning the new `bio` data

In [81]:
bio = tracks['bio'].apply(pd.Series)
bio.head()
#artist_tag['tag'] = artist_tag['tag'].apply(pd.Series)

,links,published,summary,content,0
0,"{'link': {'#text': '', 'rel': 'original', 'hre...","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",NaN
1,"{'link': {'#text': '', 'rel': 'original', 'hre...","25 Mar 2006, 06:09","Aubrey Drake Graham (born October 24, 1986) is...","Aubrey Drake Graham (born October 24, 1986) is...",NaN
2,"{'link': {'#text': '', 'rel': 'original', 'hre...","09 Jan 2011, 12:41","Abel Makkonen Tesfaye, popularly known as The ...","Abel Makkonen Tesfaye, popularly known as The ...",NaN
3,"{'link': {'#text': '', 'rel': 'original', 'hre...","13 Nov 2013, 05:27",A production duo featuring Andrew Taggart and ...,A production duo featuring Andrew Taggart and ...,NaN
4,"{'link': {'#text': '', 'rel': 'original', 'hre...","18 Jul 2014, 22:40",Rae Sremmurd ( /ˈreɪ ʃrˈɪmɜːrd/) is an America...,Rae Sremmurd ( /ˈreɪ ʃrˈɪmɜːrd/) is an America...,NaN


Cleaning the `artist_tag` column

In [82]:
artist_tag = tracks['artist_tag'].apply(pd.Series)
new_columns = []
for item in artist_tag.columns.to_list():
    new_columns.append('aritist_genre_'+str(item))
artist_tag.columns = new_columns
#print(artist_tag.columns.to_list(), new_columns)
artist_tag.head()

C:\Users\annas\AppData\Local\Temp\ipykernel_4728\2079752897.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  artist_tag = tracks['artist_tag'].apply(pd.Series)


,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4
0,"{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'trap', 'url': 'https://www.last.fm/t...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'hip hop', 'url': 'https://www.last.f...","{'name': 'atlanta', 'url': 'https://www.last.f..."
1,"{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'rnb', 'url': 'https://www.last.fm/ta...","{'name': 'hip hop', 'url': 'https://www.last.f...","{'name': 'Canadian', 'url': 'https://www.last...."
2,"{'name': 'rnb', 'url': 'https://www.last.fm/ta...","{'name': 'electronic', 'url': 'https://www.las...","{'name': 'dubstep', 'url': 'https://www.last.f...","{'name': 'Canadian', 'url': 'https://www.last....","{'name': 'prog-rnb', 'url': 'https://www.last...."
3,"{'name': 'electronic', 'url': 'https://www.las...","{'name': 'pop', 'url': 'https://www.last.fm/ta...","{'name': 'House', 'url': 'https://www.last.fm/...","{'name': 'electro house', 'url': 'https://www....","{'name': 'dance', 'url': 'https://www.last.fm/..."
4,"{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'hip hop', 'url': 'https://www.last.f...","{'name': 'trap', 'url': 'https://www.last.fm/t...","{'name': 'seen live', 'url': 'https://www.last..."


In [83]:
# cleaning unnecesary dictionaries
for index, row in artist_tag.iterrows():
    for column in artist_tag.columns.tolist():
        try:
            row[column] = list(row[column].values())[0] 
            #print(row[column])
        except:
            continue
artist_tag.head()

,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4
0,Hip-Hop,trap,rap,hip hop,atlanta
1,Hip-Hop,rap,rnb,hip hop,Canadian
2,rnb,electronic,dubstep,Canadian,prog-rnb
3,electronic,pop,House,electro house,dance
4,Hip-Hop,rap,hip hop,trap,seen live


Cleaning `track_tag` column:

In [84]:
track_tag = tracks['track_tag'].apply(pd.Series)
new_columns = []
for item in track_tag.columns.to_list():
    new_columns.append('track_genre_'+str(item))
track_tag.columns = new_columns
track_tag.head()

C:\Users\annas\AppData\Local\Temp\ipykernel_4728\2338087104.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  track_tag = tracks['track_tag'].apply(pd.Series)


,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
0,"{'name': 'trap', 'url': 'https://www.last.fm/t...","{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': '2010s', 'url': 'https://www.last.fm/...","{'name': 'meme rap', 'url': 'https://www.last...."
1,"{'name': 'trap', 'url': 'https://www.last.fm/t...","{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'hiphop', 'url': 'https://www.last.fm...","{'name': 'rnb', 'url': 'https://www.last.fm/ta..."
2,"{'name': 'rnb', 'url': 'https://www.last.fm/ta...","{'name': 'electronic', 'url': 'https://www.las...","{'name': 'pop', 'url': 'https://www.last.fm/ta...","{'name': '2010s', 'url': 'https://www.last.fm/...","{'name': '2016', 'url': 'https://www.last.fm/t..."
3,"{'name': '2016', 'url': 'https://www.last.fm/t...","{'name': 'electronic', 'url': 'https://www.las...","{'name': 'future bass', 'url': 'https://www.la...","{'name': 'halsey', 'url': 'https://www.last.fm...","{'name': 'electropop', 'url': 'https://www.las..."
4,"{'name': 'trap', 'url': 'https://www.last.fm/t...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'pop rap', 'url': 'https://www.last.f...","{'name': 'Southern Hip Hop', 'url': 'https://w...","{'name': 'Hip-Hop', 'url': 'https://www.last.f..."


In [85]:
# cleaning unnecesary dictionaries
for index, row in track_tag.iterrows():
    for column in track_tag.columns.tolist():
        try:
            row[column] = list(row[column].values())[0] 
        except:
            continue
track_tag.head()

,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
0,trap,Hip-Hop,rap,2010s,meme rap
1,trap,Hip-Hop,rap,hiphop,rnb
2,rnb,electronic,pop,2010s,2016
3,2016,electronic,future bass,halsey,electropop
4,trap,rap,pop rap,Southern Hip Hop,Hip-Hop


Merging the clean columns to the original dataframe and removing unnecesary columns:

In [86]:
tracks_final = pd.concat([tracks, bio, artist_tag, track_tag], axis = 1)
tracks_final.head()

,url,artist,track,artist_0,artist_1,artist_2,bio,artist_tag,track_tag,published_track,links,published,summary,content,0,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","30 Jan 2018, 04:41","{'link': {'#text': '', 'rel': 'original', 'hre...","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",NaN,Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","27 Oct 2016, 02:34","{'link': {'#text': '', 'rel': 'original', 'hre...","25 Mar 2006, 06:09","Aubrey Drake Graham (born October 24, 1986) is...","Aubrey Drake Graham (born October 24, 1986) is...",NaN,Hip-Hop,rap,rnb,hip hop,Canadian,trap,Hip-Hop,rap,hiphop,rnb
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,the weeknd,daft punk,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'rnb', 'url': 'https://www.last.fm/t...","[{'name': 'rnb', 'url': 'https://www.last.fm/t...","23 Sep 2016, 01:07","{'link': {'#text': '', 'rel': 'original', 'hre...","09 Jan 2011, 12:41","Abel Makkonen Tesfaye, popularly known as The ...","Abel Makkonen Tesfaye, popularly known as The ...",NaN,rnb,electronic,dubstep,Canadian,prog-rnb,rnb,electronic,pop,2010s,2016
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,the chainsmokers,halsey,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'electronic', 'url': 'https://www.la...","[{'name': '2016', 'url': 'https://www.last.fm/...","31 Jul 2016, 22:59","{'link': {'#text': '', 'rel': 'original', 'hre...","13 Nov 2013, 05:27",A production duo featuring Andrew Taggart and ...,A production duo featuring Andrew Taggart and ...,NaN,electronic,pop,House,electro house,dance,2016,electronic,future bass,halsey,electropop
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,rae sremmurd,gucci mane,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","27 Oct 2016, 05:23","{'link': {'#text': '', 'rel': 'original', 'hre...","18 Jul 2014, 22:40",Rae Sremmurd ( /ˈreɪ ʃrˈɪmɜːrd/) is an America...,Rae Sremmurd ( /ˈreɪ ʃrˈɪmɜːrd/) is an America...,NaN,Hip-Hop,rap,hip hop,trap,seen live,trap,rap,pop rap,Southern Hip Hop,Hip-Hop


In [87]:
tracks_final.drop(['bio', 'artist_tag', 'track_tag', 'links', 0], axis=1, inplace=True)
tracks_final.head()

,url,artist,track,artist_0,artist_1,artist_2,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,None,None,"27 Oct 2016, 02:34","25 Mar 2006, 06:09","Aubrey Drake Graham (born October 24, 1986) is...","Aubrey Drake Graham (born October 24, 1986) is...",Hip-Hop,rap,rnb,hip hop,Canadian,trap,Hip-Hop,rap,hiphop,rnb
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,the weeknd,daft punk,None,"23 Sep 2016, 01:07","09 Jan 2011, 12:41","Abel Makkonen Tesfaye, popularly known as The ...","Abel Makkonen Tesfaye, popularly known as The ...",rnb,electronic,dubstep,Canadian,prog-rnb,rnb,electronic,pop,2010s,2016
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,the chainsmokers,halsey,None,"31 Jul 2016, 22:59","13 Nov 2013, 05:27",A production duo featuring Andrew Taggart and ...,A production duo featuring Andrew Taggart and ...,electronic,pop,House,electro house,dance,2016,electronic,future bass,halsey,electropop
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,rae sremmurd,gucci mane,None,"27 Oct 2016, 05:23","18 Jul 2014, 22:40",Rae Sremmurd ( /ˈreɪ ʃrˈɪmɜːrd/) is an America...,Rae Sremmurd ( /ˈreɪ ʃrˈɪmɜːrd/) is an America...,Hip-Hop,rap,hip hop,trap,seen live,trap,rap,pop rap,Southern Hip Hop,Hip-Hop


# Adding new columns

Generating a new clean genres column based on the extracted ones from de apis.

In [88]:
tracks_final.head(1)

,url,artist,track,artist_0,artist_1,artist_2,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


In [89]:
tracks_final['music_genre'] = np.where(tracks_final['aritist_genre_0'].isnull() == True, tracks_final['track_genre_0'].str.lower(), tracks_final['aritist_genre_0'].str.lower())
tracks_final['music_genre'].value_counts()

hip-hop         3418
pop             1841
rap             1099
country          880
rnb              787
                ... 
southern rap       1
deutschrap         1
noise              1
rap francais       1
noise rock         1
Name: music_genre, Length: 381, dtype: int64

In [90]:
count_genres = Counter(genres for genres in tracks_final['music_genre'])
print(f"There are {len(count_genres)} different music genres.")

There are 796 different music genres.


In [91]:
dict_genres = dict(count_genres.most_common(25))

In [92]:
fuzz.ratio('hip-hop', "hip hop")

86

In [93]:
tracks_final["clean_music_genre"] = tracks_final.apply(lambda x: sa.music_genres(x["music_genre"], dict_genres), axis = 1)
tracks_final["clean_music_genre"].value_counts()

hip-hop              3431
other                2096
pop                  1841
rap                  1102
country               900
rnb                   787
electronic            718
trap                  666
indie                 416
rock                  330
soul                  322
indie pop             241
k-pop                 188
house                 183
reggaeton             165
jazz                  164
seen live             142
female vocalists      136
black metal           135
classic rock          126
folk                  122
hip hop               108
latin                 102
cloud rap              90
acoustic               76
singer-songwriter      72
Name: clean_music_genre, dtype: int64

Adding gender info column.

In [94]:
tracks_final.head(1)

,url,artist,track,artist_0,artist_1,artist_2,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,music_genre,clean_music_genre
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,hip-hop,hip-hop


In [95]:
tracks_final["gender"] = tracks_final['summary'].apply(sa.generos)

In [96]:
tracks_final['gender'].value_counts() / tracks_final.shape[0]

male          0.547036
group         0.167815
female        0.158196
non-binary    0.016986
Name: gender, dtype: float64

Age from artist

-- taking the data from the summary

In [97]:
tracks_final['birthday'] =  tracks_final['content'].apply(lambda x: re.findall(r'\w{1,} \d{1,2}, \d{4}|\d{1,2} \w{4,} \d{4}',str(x))).str[0]
tracks_final['birthday'].value_counts()

October 24, 1986      166
November 20, 1983     160
June 18, 1999         131
December 2, 1998      126
January 22, 1990      120
                     ... 
September 16, 1964      1
October 29, 2018        1
July 30, 2021           1
December 12, 2014       1
1 March 1995            1
Name: birthday, Length: 1324, dtype: int64

In [98]:
tracks_final[tracks_final['gender'] != 'group']['birthday'].isnull().sum()

4573

In [99]:
tracks_final[[1, 2, 3]] = tracks_final['birthday'].str.split(' ', expand=True)
tracks_final.head(1)

,url,artist,track,artist_0,artist_1,artist_2,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,music_genre,clean_music_genre,gender,birthday,1,2,3
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,hip-hop,hip-hop,group,"June 15, 2013",June,"15,",2013


In [100]:
tracks_final['month_text'] = ''
tracks_final['day'] = ''
for index, row in tqdm(tracks_final.iterrows(), total = tracks_final.shape[0]):
    try:
        if ',' in row[2]:
            #print('str')
            row['month_text'] = row[1]
            row['day'] = row[2].replace(',', '')
    except:
        if row[2] == np.nan:
            #print('nan')
            row['month_text'] = np.nan
            row['day'] = np.nan
        else:
            #print('float')
            row['month_text'] = row[2]
            row['day'] = row[1]

100%|██████████| 14659/14659 [00:04<00:00, 3549.97it/s]


In [101]:
tracks_final['month_text'].value_counts()

             1015
October       858
June          830
May           783
September     775
January       741
November      656
December      611
April         598
August        577
July          548
March         536
February      403
Sep             2
Dec             2
died            1
and             1
Oct             1
Name: month_text, dtype: int64

In [102]:
tracks_final.head(1)

,url,artist,track,artist_0,artist_1,artist_2,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,music_genre,clean_music_genre,gender,birthday,1,2,3,month_text,day
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,hip-hop,hip-hop,group,"June 15, 2013",June,"15,",2013,June,15


In [103]:
tracks_final["month"] = tracks_final["month_text"].apply(sa.month_as_number)
tracks_final["month"].value_counts()

10.0    858
6.0     830
5.0     783
9.0     775
1.0     741
11.0    656
12.0    611
4.0     598
8.0     577
7.0     548
3.0     536
2.0     403
Name: month, dtype: int64

In [104]:
tracks_final['birthday_date'] = tracks_final.apply(lambda x: sa.date_conversion(x[3], x["month"], x["day"]), axis = 1)

In [105]:
tracks_final['birthday_date'].value_counts()

1986-10-24    166
1983-11-20    160
1999-06-18    131
1998-12-02    126
1990-01-22    120
             ... 
1984-09-19      1
1995-09-05      1
2005-05-24      1
1988-07-21      1
1972-08-30      1
Name: birthday_date, Length: 1057, dtype: int64

In [106]:
tracks_final['age'] = tracks_final['birthday_date'].apply(sa.calculate_age)
tracks_final['age'].value_counts()

23.0     412
30.0     406
24.0     402
32.0     375
27.0     350
        ... 
112.0      1
148.0      1
94.0       1
93.0       1
103.0      1
Name: age, Length: 109, dtype: int64

Cleaning unnecesary columns

In [107]:
tracks_final.head(1)

,url,artist,track,artist_0,artist_1,artist_2,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,music_genre,clean_music_genre,gender,birthday,1,2,3,month_text,day,month,birthday_date,age
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,hip-hop,hip-hop,group,"June 15, 2013",June,"15,",2013,June,15,6.0,2013-06-15,9.0


In [108]:
tracks_final.drop(['birthday',	1,	2,	3,	'month_text',	'day',	'month', 'music_genre'], axis =1, inplace=True)
tracks_final.rename({'clean_music_genre':'music_genre'}, axis = 1, inplace=True)

# EDA

Total number of rows and columns of the dataframe

In [109]:
tracks_final.shape

(14659, 24)

Counting duplicated rows

In [110]:
tracks_final.duplicated().sum()

0

Counting null values

In [111]:
tracks_final.isnull().sum() / tracks_final.shape[0]

url                0.000000
artist             0.000000
track              0.000000
artist_0           0.000000
artist_1           0.828297
artist_2           0.954772
published_track    0.634423
published          0.000136
summary            0.000136
content            0.000136
aritist_genre_0    0.028447
aritist_genre_1    0.041681
aritist_genre_2    0.049935
aritist_genre_3    0.062487
aritist_genre_4    0.073402
track_genre_0      0.663756
track_genre_1      0.678900
track_genre_2      0.692885
track_genre_3      0.707006
track_genre_4      0.721195
music_genre        0.000000
gender             0.109967
birthday_date      0.459990
age                0.459990
dtype: float64

In [112]:
tracks_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14659 entries, 0 to 454549
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   url              14659 non-null  object        
 1   artist           14659 non-null  object        
 2   track            14659 non-null  object        
 3   artist_0         14659 non-null  object        
 4   artist_1         2517 non-null   object        
 5   artist_2         663 non-null    object        
 6   published_track  5359 non-null   object        
 7   published        14657 non-null  object        
 8   summary          14657 non-null  object        
 9   content          14657 non-null  object        
 10  aritist_genre_0  14242 non-null  object        
 11  aritist_genre_1  14048 non-null  object        
 12  aritist_genre_2  13927 non-null  object        
 13  aritist_genre_3  13743 non-null  object        
 14  aritist_genre_4  13583 non-null  obje

The only null columns available on the dataset are on the `streams` column. This one I will enrich it with api info.

Main statistic values for numeric columns

In [113]:
tracks_final.describe().T

,count,mean,std,min,25%,50%,75%,max
age,7916.0,28.225619,17.398975,0.0,21.0,28.0,35.0,148.0


Main statistic values for categorical columns

In [114]:
tracks_final.describe(include='object').T

,count,unique,top,freq
url,14659,14650,https://open.spotify.com/track/3Yt2ph8Ko0JBANp...,2
artist,14659,5211,taylor swift,200
track,14659,11635,intro,14
artist_0,14659,3866,taylor swift,202
artist_1,2517,1264,travis scott,40
artist_2,663,455,quavo,20
published_track,5359,4138,"07 May 2021, 15:50",7
published,14657,2871,"01 Jan 1970, 00:00",1092
summary,14657,3865,Taylor Alison Swift is an American singer-song...,202
content,14657,2939,,1092


# Export

In [115]:
with open('../data/pickle/lastfm.pickle', 'wb') as data_lastfm:
    pickle.dump(tracks_final, data_lastfm)